In [1]:
import torch
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForTokenClassification

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy
import ast
import cv2
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained model tokenizer (vocabulary)
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")



C:\Miniconda\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def verify_tokens(_tokenizer):
    tensor = _tokenizer.encode("How is life barack obama abcdde wuhan china 2020", return_tensors = "pt", add_special_tokens = True)
    for _tensor in tensor.view(-1):
        print(_tensor, _tokenizer.decode(_tensor))
    
verify_tokens(tokenizer)

tensor(101) [CLS]
tensor(1731) How
tensor(1110) is
tensor(1297) life
tensor(2927) bar
tensor(8741) ##ack
tensor(184) o
tensor(2822) ##ba
tensor(1918) ##ma
tensor(170) a
tensor(1830) ##b
tensor(1665) ##c
tensor(13976) ##dd
tensor(1162) ##e
tensor(192) w
tensor(23698) ##uh
tensor(1389) ##an
tensor(5144) chin
tensor(1161) ##a
tensor(12795) 2020
tensor(102) [SEP]


In [3]:
import pandas as pd
train = pd.read_parquet("../../Dataset/train.parquet")
valid = pd.read_parquet("../../Dataset/valid.parquet")

In [4]:
train.head(3)

,OCR,image,hero,villain,victim,other,_Hero,_Villain,_Victim,_Other,Target,All_Entities
0,Bernie Elizabeth? Be informed.Compare issues m...,covid_memes_18.png,[],[],[],"[bernie sanders, elizabeth warren]",,,,[BOTH] bernie sanders [SEP] elizabeth warren [...,[BOTH] bernie sanders [SEP] elizabeth warren [...,"[bernie sanders, elizabeth warren]"
1,Extending Brexit deadline October 31st order e...,covid_memes_19.png,[],[uk government],[],[],,[BVIL] uk government [EVIL],,,[BVIL] uk government [EVIL],[uk government]
2,kwai gkwa 0964 #nnevvy applause Thais Hong Kon...,covid_memes_252.png,[thais],[],[],[hong kong],[BH] thais [EH],,,[BOTH] hong kong [EOTH],[BH] thais [EH] [BOTH] hong kong [EOTH],"[thais, hong kong]"


In [5]:
models.resnet18(pretrained=True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
# Get Image features
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

checkpoint = torch.load("../../Models/img_class_ed.pt")
class_names = checkpoint["class_names"]

model_resnet = models.resnet18(pretrained=True)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, len(class_names))

for param in model_resnet.parameters():
    param.requires_grad = False

model_resnet.load_state_dict(checkpoint['model_state_dict'])
model_resnet.eval()
model_resnet.layer4.register_forward_hook(get_activation('layer4'))

model_resnet = model_resnet.to(device)


def getImageFeatures(imageName):
 
    transfomations = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256, 256)),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4372, 0.4457, 0.4807],[0.3319, 0.3318, 0.3384])
    ])
    if imageName.lower().startswith("covid"):
        root = "../../Dataset/covid/images"
    else:
        root = "../../Dataset/politics/images"
    img_name = os.path.join(root, imageName)
    image = cv2.imread(img_name)
    image = transfomations(image)
    image = image.to(device)
    image = image.unsqueeze(0)
    outputs = model_resnet(image)
    _, preds = torch.max(outputs, 1)

    imagePrediction = class_names.loc[class_names.Labels == preds[0].item(), "LabelName"].values[0]
    return activation["layer4"], imagePrediction


In [7]:
image_feats_dict = {}
def add_imageFeatures(ds, phase):
    image_feats_dict[phase] = {}
    data = train.copy()
    for _index in data.index.values:
        sample = data.loc[_index]
        id = sample["index"]
        image = sample["image"]
        # Get image features
        _image_feats, prediction = getImageFeatures(image)

        image_feats_dict[phase][id] = (_image_feats.cpu(),prediction)

train = train.reset_index()
valid = valid.reset_index()
add_imageFeatures(train, "train")
add_imageFeatures(valid, "valid")

In [8]:
# Explode dataset based on category column and merge it
def getexplodedds(ds):
    data = ds.copy()
    df = pd.DataFrame()
    for idx,col in enumerate(["hero","villain","victim","other"]):
        _df = data[["index","OCR",col]].explode(col)
        _df = _df[~_df[col].isna()]
        _df.columns = ["ID","OCR","Entity"]
        _df["Target"] = idx
        _df["TargetLabel"] = col
        df = pd.concat([df,_df], axis = 0, ignore_index = True)

    return df

train_df = getexplodedds(train)
valid_df = getexplodedds(valid)

In [9]:
train_df[train_df["ID"] == 0].head(3)

,ID,OCR,Entity,Target,TargetLabel
3812,0,Bernie Elizabeth? Be informed.Compare issues m...,bernie sanders,3,other
3813,0,Bernie Elizabeth? Be informed.Compare issues m...,elizabeth warren,3,other


In [10]:
# resample skewed classes
def resample(ds, target, repeat_freq):
    data = ds.copy()
    _df = data.loc[data.TargetLabel == target].copy()
    for idx in range(repeat_freq):
        data = pd.concat([data, _df], axis = 0, ignore_index=True)
    return data

train_df = resample(train_df, "hero", 10)
train_df = resample(train_df, "victim", 6)
train_df = resample(train_df, "villain", 3)

train_df.TargetLabel.value_counts()

other      13702
villain     9708
victim      6370
hero        5225
Name: TargetLabel, dtype: int64

In [11]:
train_df = train_df.sample(frac=1).reset_index(drop = True)
train_df.head(3)

,ID,OCR,Entity,Target,TargetLabel
0,1526,Government: work home Fishermen: Working home,work from home,3,other
1,4432,THE REPUBLICAN PARTY'S PLEDGE TO THE MIDDLE CL...,poor,2,victim
2,4834,WE CAN NOT ALLOW SYRIANS TO BOMB SYRIA BECAUSE...,syrians,2,victim


In [12]:
# Tweet length distribution
import plotly.express as px
tweet_len = list(map(len, train_df.OCR.values))
px.histogram(tweet_len)

In [13]:
tweet_len = list(map(len, train_df.Entity.values))
px.histogram(tweet_len)

In [14]:
def getDataLoader(data, batch_size, phase):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids_sent = []
    attention_masks_sent = []

    input_ids_ent = []
    attention_masks_ent = []

    image_embeddings = []
    # For every sentence...
    for id, sent, entity in zip(data.ID.values, data.OCR.values, data.Entity.values):
        image_embeddings.append(image_feats_dict[phase][id][0])
        for _entity in [entity]:
            # `encode_plus` will:
            #   (1) Tokenize the sentence.
            #   (2) Prepend the `[CLS]` token to the start.
            #   (3) Append the `[SEP]` token to the end.
            #   (4) Map tokens to their IDs.
            #   (5) Pad or truncate the sentence to `max_length`
            #   (6) Create attention masks for [PAD] tokens.
            encoded_dict = tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 100,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                        )

            encoded_dict_ent = tokenizer.encode_plus(
                                _entity,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 20,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                        )
            
            # Add the encoded sentence to the list.    
            input_ids_sent.append(encoded_dict['input_ids'])
            input_ids_ent.append(encoded_dict_ent['input_ids'])
            
            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks_sent.append(encoded_dict['attention_mask'])
            attention_masks_ent.append(encoded_dict_ent['attention_mask'])

    # Convert the lists into tensors.
    input_ids_sent = torch.cat(input_ids_sent, dim=0)
    attention_masks_sent = torch.cat(attention_masks_sent, dim=0)

    input_ids_ent = torch.cat(input_ids_ent, dim=0)
    attention_masks_ent = torch.cat(attention_masks_ent, dim=0)
    labels = torch.tensor(data.Target.values)

    image_embeddings = torch.cat(image_embeddings, dim = 0)    
    ids = torch.tensor(data.ID.values)
    
    # Create the DataLoader.
    prediction_data = TensorDataset(ids, input_ids_sent, attention_masks_sent, input_ids_ent, attention_masks_ent, labels, image_embeddings)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler
    , batch_size=batch_size)

    return prediction_dataloader


In [15]:
# Set the batch size.  
batch_size = 32  
print(train_df.shape)

dataloaders = {}
dataloaders["train"] = getDataLoader(train_df,batch_size, "train")
dataloaders["valid"] = getDataLoader(valid_df,batch_size, "valid")

dataset_sizes = {}
dataset_sizes["train"] = len(train_df)
dataset_sizes["valid"] = len(valid_df)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(35005, 5)


C:\Miniconda\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



In [16]:
class OCRClassification(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Sentence
        self.s_conv1 = nn.Conv2d(in_channels=100, out_channels=64, kernel_size=(1,10), padding='same')
        self.s_maxpool = nn.MaxPool2d(kernel_size=(1,5))
        self.s_conv2 = nn.Conv2d(64, 64, (1,10),padding='same')
        
        # Entity
        self.e_conv1 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=(1,10), padding='same')
        self.e_maxpool = nn.MaxPool2d(kernel_size=(1,5))
        self.e_conv2 = nn.Conv2d(32, 64, (1,10),padding='same')

        # Image
        self.i_conv1 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(1,1), padding='same')
        self.i_conv2 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=(1,1), padding='same')
        self.i_conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(1,1), padding='same')

        # Agg
        self.conv1 = nn.Conv2d(in_channels=192, out_channels=128, kernel_size=(1,10), padding='same')
        self.maxpool = nn.MaxPool2d(kernel_size=(1,2))
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(1,10), padding='same')
        self.fc1 = nn.Linear(64*12, 4)
        self.dropout = nn.Dropout(0.2)

    def forward(self, sent, entity, image):
        s_x = self.dropout(self.s_maxpool(F.relu(self.s_conv1(sent))))
        s_x = self.dropout(self.s_maxpool(F.relu(self.s_conv2(s_x))))

        e_x = self.dropout(self.e_maxpool(F.relu(self.e_conv1(entity))))
        e_x = self.dropout(self.e_maxpool(F.relu(self.e_conv2(e_x))))

        i_x = self.dropout(F.relu(self.i_conv1(image)))
        i_x = self.dropout(F.relu(self.i_conv2(i_x)))
        i_x = self.dropout(F.relu(self.i_conv3(i_x)))
        i_x = i_x.view(i_x.size(0), i_x.size(1),1, -1)


        input = torch.cat([F.pad(s_x, (0, 9)), F.pad(e_x, (0, 9)), i_x], dim=1)

        x = self.dropout(self.maxpool(F.relu(self.conv1(input))))
        x = self.dropout(self.maxpool(F.relu(self.conv2(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

In [17]:
def train_model(model, bertmodel, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for ids, b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent, labels, image_embeddings in dataloaders[phase]:
                # Get output from bert model
                b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent= b_input_ids_sent.to(device), b_input_mask_sent.to(device), b_input_ids_ent.to(device), b_input_mask_ent.to(device)
                
                image_embeddings = image_embeddings.to(device)
                outputs_sent = bertmodel(b_input_ids_sent, token_type_ids=None, 
                                attention_mask=b_input_mask_sent)

                outputs_ent = bertmodel(b_input_ids_ent, token_type_ids=None, 
                                attention_mask=b_input_mask_ent)

                # Get hidden states from output
                hidden_states_sent = outputs_sent.hidden_states
                hidden_states_ent = outputs_ent.hidden_states

                # Stack all token embeddings from bert model
                token_embeddings_sent = torch.stack(hidden_states_sent, dim=0)
                token_embeddings_ent = torch.stack(hidden_states_ent, dim=0)

                # Take second last layer from hidden state and take a mean across tokens to get sentence level encoding
                # This will give an output of shape (batch_size * 768)
                sentence_embedding =token_embeddings_sent[-2,...].unsqueeze(dim = 2)
                entity_embedding = token_embeddings_ent[-2,...].unsqueeze(dim = 2)

                #inputs = torch.cat([sentence_embedding, entity_embedding], dim=1).unsqueeze(dim=1).to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(sentence_embedding, entity_embedding, image_embeddings )
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * sentence_embedding.size(0)
                running_corrects += torch.sum(preds == labels.data)
            #if phase == 'train':
            #    scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': best_model_wts,
                            'loss': loss,
                            }, "../../Models/ocr_class.pt")

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [18]:
# test
'''
bertmodel = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER", output_hidden_states=True)

sent = "how is baraack obama"
encoded_dict = tokenizer.encode_plus(
                                sent,                      # Sentence to encode.
                                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                max_length = 200,           # Pad & truncate all sentences.
                                pad_to_max_length = True,
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                        )
outputs_sent = bertmodel(encoded_dict["input_ids"], token_type_ids=None, 
                                attention_mask=encoded_dict["attention_mask"])

# Get hidden states from output
hidden_states_sent = outputs_sent.hidden_states

# Stack all token embeddings from bert model
token_embeddings_sent = torch.stack(hidden_states_sent, dim=0)
'''


'\nbertmodel = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER", output_hidden_states=True)\n\nsent = "how is baraack obama"\nencoded_dict = tokenizer.encode_plus(\n                                sent,                      # Sentence to encode.\n                                add_special_tokens = True, # Add \'[CLS]\' and \'[SEP]\'\n                                max_length = 200,           # Pad & truncate all sentences.\n                                pad_to_max_length = True,\n                                return_attention_mask = True,   # Construct attn. masks.\n                                return_tensors = \'pt\',     # Return pytorch tensors.\n                        )\noutputs_sent = bertmodel(encoded_dict["input_ids"], token_type_ids=None, \n                                attention_mask=encoded_dict["attention_mask"])\n\n# Get hidden states from output\nhidden_states_sent = outputs_sent.hidden_states\n\n# Stack all token embeddings from bert mode

In [28]:
def test_model(model, bertmodel, phase):
    results = []
    model.eval()   # Set model to evaluate mode

    with torch.set_grad_enabled(False):
    # Iterate over data.
        for ids, b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent, labels, image_embeddings in dataloaders[phase]:
            # Get output from bert model
            b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent= b_input_ids_sent.to(device), b_input_mask_sent.to(device), b_input_ids_ent.to(device), b_input_mask_ent.to(device)
            
            image_embeddings = image_embeddings.to(device)
            outputs_sent = bertmodel(b_input_ids_sent, token_type_ids=None, 
                            attention_mask=b_input_mask_sent)

            outputs_ent = bertmodel(b_input_ids_ent, token_type_ids=None, 
                            attention_mask=b_input_mask_ent)

            # Get hidden states from output
            hidden_states_sent = outputs_sent.hidden_states
            hidden_states_ent = outputs_ent.hidden_states

            # Stack all token embeddings from bert model
            token_embeddings_sent = torch.stack(hidden_states_sent, dim=0)
            token_embeddings_ent = torch.stack(hidden_states_ent, dim=0)

            # Take second last layer from hidden state and take a mean across tokens to get sentence level encoding
            # This will give an output of shape (batch_size * 768)
            sentence_embedding =token_embeddings_sent[-2,...].unsqueeze(dim = 2)
            entity_embedding = token_embeddings_ent[-2,...].unsqueeze(dim = 2)

            #inputs = torch.cat([sentence_embedding, entity_embedding], dim=1).unsqueeze(dim=1).to(device)
            labels = labels.to(device)

            # forward
            # track history if only in train
            outputs = model(sentence_embedding, entity_embedding, image_embeddings )
            _, preds = torch.max(outputs, 1)
            results += list(zip(ids.numpy(), labels.cpu().numpy(), preds.cpu().numpy()))
    
    return results


### Training

In [23]:
model_ocr = OCRClassification()
if os.path.exists("../../Models/ocr_class.pt"):
    checkpoint = torch.load("../../Models/ocr_class.pt")
    model_ocr.load_state_dict(checkpoint['model_state_dict'])
    
model_ocr = model_ocr.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ocr.parameters(), lr=0.001) #, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

#bertmodel = BertModel.from_pretrained('bert-base-uncased'
#                                       ,output_hidden_states = True, # Whether the model returns all hidden-states.
#                                  )

bertmodel = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER", output_hidden_states=True)
for param in bertmodel.parameters():
    param.requires_grad = False

bertmodel = bertmodel.to(device)
bertmodel.eval()

model_ft = train_model(model_ocr, bertmodel, criterion, optimizer_ft, None,
                       num_epochs=20)                                 

Epoch 0/19
----------
train Loss: 0.6162 Acc: 0.7471
valid Loss: 1.2643 Acc: 0.5950

Epoch 1/19
----------
train Loss: 0.6051 Acc: 0.7499
valid Loss: 1.2343 Acc: 0.5998

Epoch 2/19
----------
train Loss: 0.6026 Acc: 0.7520
valid Loss: 1.2151 Acc: 0.5979

Epoch 3/19
----------
train Loss: 0.5891 Acc: 0.7582
valid Loss: 1.2396 Acc: 0.6008

Epoch 4/19
----------
train Loss: 0.5819 Acc: 0.7599
valid Loss: 1.2262 Acc: 0.5930

Epoch 5/19
----------
train Loss: 0.5754 Acc: 0.7633
valid Loss: 1.2335 Acc: 0.6080

Epoch 6/19
----------
train Loss: 0.5669 Acc: 0.7679
valid Loss: 1.2470 Acc: 0.6148

Epoch 7/19
----------
train Loss: 0.5609 Acc: 0.7699
valid Loss: 1.2390 Acc: 0.6201

Epoch 8/19
----------
train Loss: 0.5509 Acc: 0.7740
valid Loss: 1.2534 Acc: 0.6201

Epoch 9/19
----------
train Loss: 0.5475 Acc: 0.7755
valid Loss: 1.3185 Acc: 0.6042

Epoch 10/19
----------
train Loss: 0.5422 Acc: 0.7792
valid Loss: 1.3064 Acc: 0.6245

Epoch 11/19
----------
train Loss: 0.5342 Acc: 0.7824
valid Loss

### Inferencing

In [25]:
model_ocr = OCRClassification()
if os.path.exists("../../Models/ocr_class.pt"):
    checkpoint = torch.load("../../Models/ocr_class.pt")
    model_ocr.load_state_dict(checkpoint['model_state_dict'])
else: 
    raise "Model does not exist"

model_ocr.eval()
model_ocr = model_ocr.to(device)


bertmodel = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER", output_hidden_states=True)

for param in bertmodel.parameters():
    param.requires_grad = False

bertmodel.eval()
bertmodel = bertmodel.to(device)
                              

In [29]:
from sklearn.metrics import classification_report
results_t = test_model(model_ocr, bertmodel, "train")  
x = pd.DataFrame(results_t, columns = ["Id", "Actual","Pred"])
print(classification_report(x.Actual.values, x.Pred.values))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5225
           1       0.78      0.92      0.85      9708
           2       0.90      0.94      0.92      6370
           3       0.95      0.79      0.86     13702

    accuracy                           0.88     35005
   macro avg       0.89      0.91      0.90     35005
weighted avg       0.89      0.88      0.88     35005



In [31]:
from sklearn.metrics import classification_report
results_v = test_model(model_ocr, bertmodel, "valid")  
x = pd.DataFrame(results_v, columns = ["Id", "Actual","Pred"])
print(classification_report(x.Actual.values, x.Pred.values))

              precision    recall  f1-score   support

           0       0.08      0.09      0.08        54
           1       0.32      0.49      0.39       305
           2       0.16      0.25      0.19       121
           3       0.85      0.72      0.78      1589

    accuracy                           0.64      2069
   macro avg       0.35      0.39      0.36      2069
weighted avg       0.71      0.64      0.67      2069

